In [9]:
%load_ext autoreload
%autoreload
%matplotlib inline

import os
import sys
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

from newscrawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. 초기화

In [10]:
crawler = NewsCrawler(storage='bigquery')

checking bigquery storage... done (23.43 seconds)
we have total 425129 articles (360885 downloaded, 64244 trashed)


# 1. URL을 모은다

In [11]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (103.79 seconds): Collecting URLs... yahoo               

100.00% (0.11 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25975,25906,1004
reuters,1094,1094,166
investing.com,931,931,108
euronews,1063,1063,102
dailymail,2885,2885,69
...,...,...,...
nextbigfuture,36,36,0
axios,132,129,0
vice,102,102,0
msnbc,136,101,0


,url,pubs,actual_pub
0,https://cnn.com/2019/10/30/business/grocery-de...,"cnn, axios",cnn
1,https://politico.com/magazine/story/2019/11/05...,"politico, realclearpolitics",politico
2,https://politico.com/magazine/story/2019/10/30...,"politico, realclearpolitics",politico
3,https://politico.com/news/2019/11/06/joe-biden...,"politico, realclearpolitics",politico
4,https://time.com/5720747/jon-meacham-trump-imp...,"time, realclearpolitics",time
...,...,...,...
64,https://nbcnews.com/politics/trump-impeachment...,"msnbc, nbcnews",nbcnews
65,https://nbcnews.com/think/why-is-this-happenin...,"msnbc, nbcnews",nbcnews
66,https://nbcnews.com/politics/donald-trump/inte...,"msnbc, nbcnews",nbcnews
67,https://nbcnews.com/politics/2020-election/rep...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 크롤링한다

In [12]:
crawl_result = crawler.crawl(); crawl_result

100.00% (356.05 seconds): Crawling... euronews            



collect                                crawl              
              collected collected_unique selected downloaded trashed total
all               25975            25906     1004        510     494  1004
reuters            1094             1094      166         64     102   166
investing.com       931              931      108         33      75   108
euronews           1063             1063      102          5      97   102
dailymail          2885             2885       69         65       4    69
...                 ...              ...      ...        ...     ...   ...
nextbigfuture        36               36        0          0       0     0
axios               132              129        0          0       0     0
vice                102              102        0          0       0     0
msnbc               136              101        0          0       0     0
arstechnica         208              208        0          0       0     0

[74 rows x 6 columns]

# 3. 뉴스를 저장한다

In [13]:
crawler.record()

Recording to bigquery storage... done (17.81 seconds)


# 4. 결과보고

In [14]:
crawler.report(using='sendgrid')

202


In [17]:
64788+982

65770